Embraer Reports

In [14]:
import xlwings as xw
import pandas as pd
import datetime
import os
from pathlib import Path
import sys
sys.path.append('..')
from unprotect_xlsx import remove_excel_protection

In [15]:
current_date = datetime.datetime.now()

def get_previous_month(date):
    first_day_of_current_month = datetime.date(date.year, date.month, 1)
    last_day_of_previous_month = first_day_of_current_month - datetime.timedelta(days=1)
    return last_day_of_previous_month.strftime('%B')

def get_previous_two_months(date):
    first_day_of_current_month = datetime.date(date.year, date.month, 1)
    last_day_of_two_months_back = first_day_of_current_month - datetime.timedelta(days=1)
    last_day_of_two_months_back = last_day_of_two_months_back.replace(day=1) - datetime.timedelta(days=1)
    return last_day_of_two_months_back.strftime('%B')

prev_month = get_previous_two_months(current_date)
month = get_previous_month(current_date)

general_path_prev = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany December 2024\1. Embraer", rf"1.- EZA1 Accounts Receivable Intercompany {prev_month.upper()} - General 2024.xlsx")
general_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany January 2025\1. Embraer", rf"1.- EZA1 Accounts Receivable Intercompany {month.upper()} - General 2025.xlsx")
general_book = xw.Book(general_path_prev, update_links=False)
general_book.save(general_path)

In [16]:
prev_month

'December'

In [17]:
screen_sheet = general_book.sheets['Screen R4 GL 4 57']

screen_sheet.range("A2").clear()

screen_sheet.range("A2").value = f"Accts Receivables {month} 2025"

screen_sheet.range("A2").api.Font.Bold = True

In [18]:
row = screen_sheet.range('C4').current_region.last_cell.row

In [19]:
screen_sheet.range(f"{row}:{row}").api.Insert()
screen_sheet.range(f"A{row}").value = f"{month.upper()} 25 ADJ."
screen_sheet.range(f"C{row+1}").formula = f"=SUM(C4:C{row})"

In [20]:
for shape in screen_sheet.shapes:
    print(shape.name)

try:
    screen_sheet.shapes['Picture 5'].delete()
    screen_sheet.shapes['Picture 6'].delete()
except KeyError:
    print("The picture you are trying to delete doesn't exist.")



Picture 1
Picture 2
Comment 1
The picture you are trying to delete doesn't exist.


In [21]:
amarre_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR", rf"1.1- AMARRE AGING Vs. BALANZA AJUSTADO {month.upper()} 2025.xlsx")
amarre_book = xw.Book(amarre_path, update_links=False)
trialb_sheet = amarre_book.sheets['TRIAL BALANCE']

In [22]:
shapes_to_use = []

# Loop through all the shapes in the sheet and store their names
for i, shape in enumerate(trialb_sheet.shapes):
    if i > 0:  # Skip the first shape (i.e., index 0)
        shapes_to_use.append(shape)

shapes_to_use

[<Shape 'Picture 4' in <Sheet [1.1- AMARRE AGING Vs. BALANZA AJUSTADO JANUARY 2025.xlsx]TRIAL BALANCE>>,
 <Shape 'Picture 5' in <Sheet [1.1- AMARRE AGING Vs. BALANZA AJUSTADO JANUARY 2025.xlsx]TRIAL BALANCE>>]

In [23]:
if len(shapes_to_use) >= 2:
    shape1 = shapes_to_use[0]  # This is the second shape in the original list
    shape2 = shapes_to_use[1]  # This is the third shape in the original list
    
    # You can now proceed with copying and pasting the shapes
    shape1.api.Copy()
    screen_sheet.api.Paste()

    pasted_shape1 = screen_sheet.shapes[-1]
    pasted_shape1.left = screen_sheet.range("B61").left
    pasted_shape1.top = screen_sheet.range("B61").top

    shape2.api.Copy()
    screen_sheet.api.Paste()

    pasted_shape2 = screen_sheet.shapes[-1]
    pasted_shape2.left = screen_sheet.range("G61").left
    pasted_shape2.top = screen_sheet.range("G61").top
else:
    print("Not enough shapes to assign to shape1 and shape2")

In [24]:
#Copy and paste the balance of the 1110
balance_1110 = trialb_sheet.range("H9").value
screen_sheet.range("C4").value = balance_1110


In [25]:
resumen_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR", rf"3.- RESUMEN AR {month.upper()} 2025.xlsx")
resume_book = xw.Book(resumen_path, update_links=False)
resume_sheet = resume_book.sheets['RESUMEN']
ar_sheet = resume_book.sheets['ACCOUNTS RECEIVABLE (AR)']

source_cell = ar_sheet.range("O74")

screen_sheet.range(f"C{row}").formula = f"='C:\\Users\\nil.monreal\\Desktop\\7. Cierre de mes AR\\[3.- RESUMEN AR JANUARY 2025.xlsx]ACCOUNTS RECEIVABLE (AR)'!{source_cell.address}"

resume_table = resume_sheet.range("B18:F26").value

last_row = screen_sheet.range('C' + str(screen_sheet.cells.last_cell.row)).end('up').row
last_row = last_row-6

screen_sheet.range(f"A{last_row}").value = resume_table

In [26]:
row_provision = row+5
row_provision

59

In [27]:
screen_sheet.range(f"C{row_provision}").formula = fr"=-SUM('C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\[3.- RESUMEN AR {month.upper()} 2025.xlsx]TRIAL BALANCE'!$J$9)"

In [28]:
ar_sheet = general_book.sheets["Accounts Receivable"]
embraer_sheet = amarre_book.sheets["Embraer"]

# Step 1: Find the last non-empty row in column F
last_row = embraer_sheet.range('F' + str(embraer_sheet.cells.last_cell.row)).end('up').row

# Step 2: Extract the range from row 4 to the last non-empty row in column F
data_range = embraer_sheet.range(f"A5:Q{last_row}").value  # Assuming you want columns A to F

# Step 3: Convert the range to a pandas DataFrame
columns = embraer_sheet.range("A4:Q4").value  # Assuming the headers are in row 3
df = pd.DataFrame(data_range, columns=columns)

print(df.Total.sum())

amarre_book.close()

15097007.510000002


In [29]:
if ar_sheet.api.ProtectContents:
    general_book.save()
    general_book.close()
    remove_excel_protection(general_path,general_path)
    general_book = xw.Book(general_path, update_links=False)
    screen_sheet = general_book.sheets['Screen R4 GL 4 57']
    ar_sheet = general_book.sheets["Accounts Receivable"]
else:
    pass
    

In [30]:
row_count = df["Tr-Date"].count()

if row_count == 0 :

    pass

elif row_count > 0:
    
    last_row = ar_sheet.range('A' + str(ar_sheet.cells.last_cell.row)).end('up').row

    ar_sheet.range(f"A2:H{last_row}").clear()

    if last_row > row_count:

        ar_sheet.range(f"A{row_count+2}:AH{last_row}").clear()
    else:

        ar_sheet.range("I2:J2").copy()
        ar_sheet.range(f"I2:J{row_count+1}").paste()

        pass

In [31]:
desired_order = ["CODE", "Ent Reference", "Vendor Code", "Customer name", "Entity", "Cust.PO", "Tr-Date", "Due-Date", "Currency", "Total", "Current", "01-30", "31-60", "61-90", "91-120", "OP", "CUENTA"]

# Reorder the DataFrame based on the new column order
df = df[desired_order]

df.insert(1, 'Trading Partner', '')
df['Trading Partner'] = 'EZA1'

cleaned_df = df.iloc[:,[0, 1, 2, 6, 7, 8, 9, 10]]

cleaned_df

ar_sheet.range("A2").value = cleaned_df.values

input("Place the snips properly and check your values are correct then click enter...")

general_book.save()
general_book.close()

In [32]:
BJGX_prev_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany December 2024\1. Embraer", rf"EZA1 Accounts Receivable Intercompany BJGX {prev_month.upper()} 2024.xlsx")
BJGX_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany January 2025\1. Embraer", rf"EZA1 Accounts Receivable Intercompany BJGX {month.upper()} 2025.xlsx")
BJGX_book = xw.Book(BJGX_prev_path, update_links=False)
BJGX_df = cleaned_df[cleaned_df['CODE']=="BJGX"]
BJGX_sheet = BJGX_book.sheets['Accounts Receivable']

if BJGX_sheet.api.ProtectContents:
    BJGX_book.save(BJGX_path)
    BJGX_book.close()
    remove_excel_protection(BJGX_path,BJGX_path)
    BJGX_book = xw.Book(general_path, update_links=False)
    BJGX_sheet = BJGX_book.sheets['Accounts Receivable']
else:
    pass

row_count = BJGX_df["Tr-Date"].count()

last_row = BJGX_sheet.range('A' + str(BJGX_sheet.cells.last_cell.row)).end('up').row

BJGX_sheet.range(f"A2:H{last_row+1}").clear()

if row_count == 0 :

    pass

elif row_count > 0:

    BJGX_sheet.range(f"A2:H{last_row}").clear()

    if last_row > row_count:

        BJGX_sheet.range(f"A{row_count+2}:AH{last_row}").clear()

    else:
        
        BJGX_sheet.range("I2:J2").copy()
        BJGX_sheet.range(f"I2:J{row_count+1}").paste()
        
        pass

    BJGX_sheet.range('A2').value = BJGX_df.values

BJGX_book.save(BJGX_path)
BJGX_book.close()

In [33]:
ESL1_prev_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany December 2024\1. Embraer", rf"EZA1 Accounts Receivable Intercompany ESL1 {prev_month.upper()} 2024.xlsx")
ESL1_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany January 2025\1. Embraer", rf"EZA1 Accounts Receivable Intercompany ESL1 {month.upper()} 2025.xlsx")
ESL1_book = xw.Book(ESL1_prev_path, update_links=False)
ESL1_df = cleaned_df[cleaned_df['CODE']=="ESL1"]
ESL1_sheet = ESL1_book.sheets['Accounts Receivable']

if ESL1_sheet.api.ProtectContents:
    ESL1_book.save(ESL1_path)
    ESL1_book.close()
    remove_excel_protection(ESL1_path,ESL1_path)
    ESL1_book = xw.Book(general_path, update_links=False)
    ESL1_sheet = ESL1_book.sheets['Accounts Receivable']
else:
    pass

row_count = ESL1_df["Tr-Date"].count()

last_row = ESL1_sheet.range('A' + str(ESL1_sheet.cells.last_cell.row)).end('up').row

ESL1_sheet.range(f"A2:H{last_row+1}").clear()

if row_count == 0 :

    pass

elif row_count > 0:

    ESL1_sheet.range(f"A2:H{last_row}").clear()

    if last_row > row_count:

        ESL1_sheet.range(f"A{row_count+2}:AH{last_row}").clear()

    else:
        
        ESL1_sheet.range("I2:J2").copy()
        ESL1_sheet.range(f"I2:J{row_count+1}").paste()

        pass

    ESL1_sheet.range('A2').value = ESL1_df.values

ESL1_book.save(ESL1_path)
ESL1_book.close()

In [34]:
LBG1_prev_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany December 2024\1. Embraer", rf"EZA1 Accounts Receivable Intercompany LBG1 {prev_month.upper()} 2024.xlsx")
LBG1_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany January 2025\1. Embraer", rf"EZA1 Accounts Receivable Intercompany LBG1 {month.upper()} 2025.xlsx")
LBG1_book = xw.Book(LBG1_prev_path, update_links=False)
LBG1_df = cleaned_df[cleaned_df['CODE']=="LBG1"]
LBG1_sheet = LBG1_book.sheets['Accounts Receivable']

if LBG1_sheet.api.ProtectContents:
    LBG1_book.save(LBG1_path)
    LBG1_book.close()
    remove_excel_protection(LBG1_path,LBG1_path)
    LBG1_book = xw.Book(general_path, update_links=False)
    LBG1_sheet = LBG1_book.sheets['Accounts Receivable']
else:
    pass

row_count = LBG1_df["Tr-Date"].count()

last_row = LBG1_sheet.range('A' + str(LBG1_sheet.cells.last_cell.row)).end('up').row

LBG1_sheet.range(f"A2:H{last_row+1}").clear()

if row_count == 0 :

    pass

elif row_count > 0:

    LBG1_sheet.range(f"A2:H{last_row}").clear()

    if last_row > row_count:

        LBG1_sheet.range(f"A{row_count+2}:AH{last_row}").clear()

    else:
        
        LBG1_sheet.range("I2:J2").copy()
        LBG1_sheet.range(f"I2:J{row_count+1}").paste()

        pass

    LBG1_sheet.range('A2').value = LBG1_df.values

LBG1_book.save(LBG1_path)
LBG1_book.close()

In [35]:
SINX_prev_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany December 2024\1. Embraer", rf"EZA1 Accounts Receivable Intercompany SINX {prev_month.upper()} 2024.xlsx")
SINX_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany January 2025\1. Embraer", rf"EZA1 Accounts Receivable Intercompany SINX {month.upper()} 2025.xlsx")
SINX_book = xw.Book(SINX_prev_path, update_links=False)
SINX_df = cleaned_df[cleaned_df['CODE']=="SINX"]
SINX_sheet = SINX_book.sheets['Accounts Receivable']

if SINX_sheet.api.ProtectContents:
    SINX_book.save(SINX_path)
    SINX_book.close()
    remove_excel_protection(SINX_path,SINX_path)
    SINX_book = xw.Book(general_path, update_links=False)
    SINX_sheet = SINX_book.sheets['Accounts Receivable']
else:
    pass

row_count = SINX_df["Tr-Date"].count()

last_row = SINX_sheet.range('A' + str(SINX_sheet.cells.last_cell.row)).end('up').row

SINX_sheet.range(f"A2:H{last_row+1}").clear()

if row_count == 0 :

    pass

elif row_count > 0:

    SINX_sheet.range(f"A2:H{last_row}").clear()

    if last_row > row_count:

        SINX_sheet.range(f"A{row_count+2}:AH{last_row}").clear()

    else:
        
        SINX_sheet.range("I2:J2").copy()
        SINX_sheet.range(f"I2:J{row_count+1}").paste()

        pass

    SINX_sheet.range('A2').value = SINX_df.values

SINX_book.save(SINX_path)
SINX_book.close()

In [36]:
SJK1_prev_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany December 2024\1. Embraer", rf"EZA1 Accounts Receivable Intercompany SJK1 {prev_month.upper()} 2024.xlsx")
SJK1_path = os.path.join(r"C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany January 2025\1. Embraer", rf"EZA1 Accounts Receivable Intercompany SJK1 {month.upper()} 2025.xlsx")
SJK1_book = xw.Book(SJK1_prev_path, update_links=False)
SJK1_df = cleaned_df[cleaned_df['CODE']=="SJK1"]
SJK1_sheet = SJK1_book.sheets['Accounts Receivable']

if SJK1_sheet.api.ProtectContents:
    SJK1_book.save(SJK1_path)
    SJK1_book.close()
    remove_excel_protection(SJK1_path,SJK1_path)
    SJK1_book = xw.Book(general_path, update_links=False)
    SJK1_sheet = SJK1_book.sheets['Accounts Receivable']
else:
    pass

row_count = SJK1_df["Tr-Date"].count()

last_row = SJK1_sheet.range('A' + str(SJK1_sheet.cells.last_cell.row)).end('up').row

SJK1_sheet.range(f"A2:H{last_row+1}").clear()

if row_count == 0 :

    pass

elif row_count > 0:

    SJK1_sheet.range(f"A2:H{last_row}").clear()

    if last_row > row_count:

        SJK1_sheet.range(f"A{row_count+2}:AH{last_row}").clear()

    else:
        
        SJK1_sheet.range("I2:J2").copy()
        SJK1_sheet.range(f"I2:J{row_count+1}").paste()

        pass

    SJK1_sheet.range('A2').value = SJK1_df.values

SJK1_book.save(SJK1_path)
SJK1_book.close()

In [37]:
input("Please verify that the job has been done properly...")

''

In [38]:
# # Path to the folder where the Excel files are located
# folder_path = Path(r'C:\Users\nil.monreal\Desktop\7. Cierre de mes AR\5.- EZA1 Accounts Receivable Itercompany January 2025\1. Embraer')

# # Find all Excel files in the folder (both .xlsx and .xls)
# files_to_delete = [file for file in folder_path.glob('*.xlsx') if prev_month.upper() in file.stem]


# # Iterate over the list and delete each file
# for excel_file in files_to_delete:
#     try:
#         excel_file.unlink()  # This will delete the file
#         print(f"Deleted: {excel_file}")
#     except FileNotFoundError:
#         print(f"File not found: {excel_file}")
#     except PermissionError:
#         print(f"Permission denied: {excel_file}")